# Python w badaniach humanistycznych
## Zajęcia 4

Zaczniemy jak poprzednio od zaciągnięcia materiałów z GitHuba.

(0a) Linuksowcy/Macowcy przechodzą terminalem do katalogu, w którym sklonowaliśmy materiały do zajęć (repozytorium o nazwie `kurs-pythona-2017Z-<login-githubowy>`) - `cd ~/tutaj/mam/ten/katalog/kurs-pythona-2017Z-piotrkasprzyk`.

(0b) Windowsowcy uruchamiają Git Bash, a potem przechodzą tamże - `cd /c/Users/ja/Pulpit/kurs-pythona-2017Z-piotrkasprzyk`.

(1) `git fetch upstream`

(2) `git checkout master`

(3) `git stash`

(4) `git merge upstream/master`

(5) `git stash pop`

Powinniśmy wzbogacić się o pliki `2017-12-19/zajecia.ipynb` (ćwiczenia na zajęciach), `2017-12-19/slajdy.ipynb` (materiały z wykładu) i `2017-12-19/praca_domowa.ipynb`.

## Korzystanie z zasobów internetowych - ciąg dalszy

## Ćwiczenie poprzednie: Znajdź identyfikator formy prawnej "kółko rolnicze" w bazie mojepanstwo.pl

In [1]:
import requests

r = requests.get('https://api-v3.mojepanstwo.pl/dane/krs_formy_prawne.json')
json_data = r.json()
#print(json_data.keys())
first_object = json_data['Dataobject'][0]
#print(first_object.keys())
#print(first_object['data'])

#for obj in json_data['Dataobject']:
#    print(obj['data']['krs_formy_prawne.nazwa'], obj['data']['krs_formy_prawne.id'])

for obj in json_data['Dataobject']:
    if 'KÓŁKO ROLNICZE' in obj['data']['krs_formy_prawne.nazwa'].upper():
        print(obj['data']['krs_formy_prawne.nazwa'], obj['data']['krs_formy_prawne.id'])

KÓŁKO ROLNICZE 5


### Spróbujmy znaleźć najstarsze Kółko Rolnicze w KRS (wg. daty rejestracji u Mojego Państwa)

In [2]:
import requests

id_kolka_rolniczego = 5

r = requests.get('https://api-v3.mojepanstwo.pl/dane/krs_podmioty.json?conditions[krs_podmioty.forma_prawna_id]={}'.format(id_kolka_rolniczego))
json_data = r.json()
print(len(json_data['Dataobject']))
print(json_data['Count'])

50
3753


### Jak widać, MojePaństwo zwraca wyniki w paczkach po 50 sztuk.

Krótka lektura https://mojepanstwo.pl/api/krs pozwoli nam zmodyfikować request tak, aby ściagał dane w paczkach po 500 sztuk (tyle się da maksymalnie).

In [3]:
import requests

id_kolka_rolniczego = 5

r = requests.get('https://api-v3.mojepanstwo.pl/dane/krs_podmioty.json?conditions[krs_podmioty.forma_prawna_id]={}&limit=500'.format(id_kolka_rolniczego))
json_data = r.json()
print(len(json_data['Dataobject']))
print(json_data['Count'])
print(json_data['Links'].keys())  # nawigacja: poprzednia strona, kolejna strona etc.

500
3753
dict_keys(['last', 'next', 'self'])


### Jak zatem ściągnąć wszystkie dane (i nie zadławić strony MojePaństwo)?
1. Pętla
2. Funkcja `sleep` z biblioteki `time`
3. Zapis do pliku (żeby nie ściągać za każdym razem jak chcemy nad nimi pracować)

In [4]:
# jeśli nie wiemy ile iteracji pętli nas czeka (ale wiemy kiedy chcemy skończyć), pętla while jest dobrym rozwiązaniem

import random

liczba_iteracji = random.randint(50, 100)  # losowa liczba całkowita z przedziału <50, 100)

suma = 0
while liczba_iteracji > 0:
    suma += liczba_iteracji
    liczba_iteracji -= 1

suma

2080

In [5]:
# funkcja sleep w module time zatrzymuje bieg programu na ustaloną liczbę sekund

import time

print('zaczynamy')
time.sleep(1)
print('minęła sekunda')
time.sleep(0.5)
print('minęło kolejne pół sekundy')

zaczynamy
minęła sekunda
minęło kolejne pół sekundy


In [6]:
# moduł json służy do konwersji json <-> pythonowe struktury

import json

s = {'lista': ['elt1', 2, 'elt3'], 'liczba': 42}
print(repr(s))  # funkcja repr() formatuje trochę inaczej, ułatwiając rozpoznawanie typów
json_s = json.dumps(s)
print(repr(json_s))
parsed_s = json.loads(json_s)
print(repr(parsed_s))

{'liczba': 42, 'lista': ['elt1', 2, 'elt3']}
'{"liczba": 42, "lista": ["elt1", 2, "elt3"]}'
{'liczba': 42, 'lista': ['elt1', 2, 'elt3']}


In [7]:
# na marginesie:
print('1')
print(1)
print(repr('2'))
print(repr(2))

1
1
'2'
2


In [8]:
import time
import json
import requests

lista_kolek_rolniczych_po_500 = 'https://api-v3.mojepanstwo.pl/dane/krs_podmioty.json?conditions[krs_podmioty.forma_prawna_id]={}&limit=500'.format(id_kolka_rolniczego)

# print(dane[0].keys())
def dump_all_pages(first_page_link, fname):
    # zerujemy plik
    open(fname, 'w', encoding='utf-8').close()
    current_link = first_page_link
    while current_link is not None:
        r = requests.get(current_link)
        print("getting: ", current_link)
        json_data = r.json()
        with open(fname, 'a', encoding='utf-8') as fp:
            for obj in json_data['Dataobject']:
                fp.write(json.dumps(obj) + '\n')
        current_link = json_data['Links']['next'] if 'next' in json_data['Links'] else None
        time.sleep(1)
        
# dump_all_pages(lista_kolek_rolniczych_po_500, '../dane/kolka_rolnicze.json')

### Żeby nie krzywdzić MojegoPaństwa na zajęciach, plik wynikowy znajduje się w materiałach
`../dane/kolka_rolnicze.json`

## Ćwiczenie 1: Wczytaj dane o kółkach rolniczych na zmienną `kolka`
z pliku `../dane/kolka_rolnicze.json`; użyj biblioteki `json`

## Ćwiczenie 2: Znajdź klucz zawierający datę rejestracji kółka

### Weźmy przykładową datę rejestacji: `'2017-12-12'`

Sortowanie jest w tym przypadku łatwe - data jest w formacie ISO 8601, w której porządek leksykograficzny odpowiada porządkowi chronologicznemu.

Potrzebujemy teraz tylko sortowania -- służy to tego funkcja `sorted`, która zwraca elementy w posorowanej kolejności

In [9]:
# prosty przypadek
for i in sorted([5, 2, 1, 3, 4]):
    print(i)

1
2
3
4
5


In [10]:
# nieprosty przypadek -- słowniki nie mają porządku, więc musimy podać klucz, po którym sortujemy
l = {'a': 5, 'b': 'a'}, {'a': 2, 'b': 'o'}, {'a': 1, 'b': 'z'}, {'a': 3, 'b': 's'}, {'a': 4, 'b': 'i'}

for i in sorted(l, key=lambda x: x['a']):
    print(i['b'])

z
o
s
i
a


## Ćwiczenie 3: Znajdź kod pocztowy najstarszego Kółka Rolniczego w KRS

## Korzystanie z zasobów internetowych - nie-JSON

Najczęstszymi danymi, na które się natkniemy, będą formaty XML (ustrukturyzowane dane) i HTML (strony internetowe, niezupełnie niepodobne do XML).

Dobrą biblioteką do parsowania jednego i drugiego jest `lxml`

In [11]:
import requests
from lxml import etree

response = requests.get('http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.01.0133%3Abook%3D1%3Acard%3D1')
root = etree.fromstring(response.content)
print(root)
print(root.tag)
print([c.tag for c in root])
print([c.tag for c in root[0]])
print([c.tag for c in root[0][0]])
print([c.tag for c in root[0][0][0]])
print(root[0][0][0][2].text)

<Element TEI.2 at 0x7ff059e38688>
TEI.2
['text']
['body']
['div1']
['milestone', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l']
οὐλομένην, ἣ μυρί᾽ Ἀχαιοῖς ἄλγε᾽ ἔθηκε,


### Parsowanie HTMLa:

In [12]:
import requests
from lxml import html
response = requests.get('https://en.wikipedia.org/wiki/Douglas_Adams')
html_root = html.fromstring(response.content)

# dalej jw
html_root.tag

'html'

### Klika wskazówek:
1. XML warto otworzyć w przeglądarce -- dostaniemy znacznie lepszy ogląd
2. HTML w zasadzie trzeba otworzyć w przeglądarce; kod (to, co widzi Python) możemy zobaczyć za pomocą "narzędzi developerskich" (F12 w większości przeglądarek)
3. Wygodnym sposobem chodzenia po XML jest XPath ( https://en.wikipedia.org/wiki/XPath ), np. `root.xpath('.//l')`

In [13]:
import requests
from lxml import etree

response = requests.get('http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.01.0133%3Abook%3D1%3Acard%3D1')
root = etree.fromstring(response.content)

print(len([l for l in root.xpath('.//l')]))  # wszystkie węzły 'l'
print(len([t for t in root.xpath('.//text()')]))  # wszystkie węzły tekstowe

txt = ''.join(t for t in root.xpath('.//text()'))  # zgodnie z powyższym -- cały tekst we wczytanym xmlu
print(txt[:100] + '...')

32
65

μῆνιν ἄειδε θεὰ Πηληϊάδεω Ἀχιλῆος
οὐλομένην, ἣ μυρί᾽ Ἀχαιοῖς ἄλγε᾽ ἔθηκε,
πολλὰς δ᾽ ἰφθίμους ψυχὰς ...


## Ćwiczenie 4: Policz, ile jest węzłow `milestone`
w pliku na stronie http://www.perseus.tufts.edu/hopper/xmlchunk?doc=Perseus%3Atext%3A1999.01.0133%3Abook%3D1%3Acard%3D1

## Następne zajęcia: 9 stycznia 2018, 16:30
### Praca domowa znajduje się w pliku `2017-12-19/praca_domowa.ipynb`
Proszę o wrzucenie jej do swojego repozytorium do 6 stycznia 2018 (sobota), do godziny 23:59.